<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Unique_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import datetime
import time
import math
import numpy as np
import scipy
from numpy import linalg as LA
import pandas as pd
import urllib.request
import seaborn as sns

In [2]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 5.0 MB/s 


In [3]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])
genes_c = genes[(pd.isna(genes.sym)) & (pd.isna(genes.chrom))]

In [4]:
expression.head()
genes.head()
samples.iloc[:20,:]

,patient,sex,region,lab,chip.version
rownames,,,,,
01_a_D_f_2.CEL,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,patient_01,female,cerebellum,Irvine,v2
01_c_M_f_1.CEL,patient_01,female,cerebellum,Michigan,v1
01_d_I_f_2.CEL,patient_01,female,D.L.P.F. cortex,Irvine,v2
01_d_M_f_1.CEL,patient_01,female,D.L.P.F. cortex,Michigan,v1
02_a_I_m_2.CEL,patient_02,male,A.C. cortex,Irvine,v2


`michigan` dataset contains the measurements, done at University of Michigan lab, of anterior cortex and dorsolateral cortex for each patient. 

`michigan_diff` subtracts measurement of anterior cortex from the dorsolateral cortex for each patient.

# Michigan

In [90]:
michigan = expression[(expression.index.str[5] == 'M') & (expression.index.str[3] != 'c') ]
michigan = michigan.sub(michigan[genes_c.index].iloc[:, -5:-1].mean(axis =1), axis = 0) 
michigan.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
01_a_M_f_1.CEL,1.595186,0.041429,-0.224280,0.783116,0.632538,0.552599,-0.158622,0.983309,2.023402,0.867365,...,0.116912,-0.091926,-0.224929,0.125358,0.252145,-0.143599,0.015697,0.063811,0.064091,7.599237
01_d_M_f_1.CEL,2.594965,0.436253,0.046795,1.256098,0.931235,1.030267,0.048435,2.127888,2.464197,2.236431,...,0.165390,-0.058486,-0.308089,0.155602,0.144891,-0.123874,0.008022,-0.028493,0.144345,7.978032
02_a_M_m_1.CEL,2.160939,0.252480,0.105039,1.121105,0.900384,1.072634,0.121465,1.652792,2.656593,2.569125,...,0.180158,0.026041,-0.255277,0.104798,0.188786,-0.073875,-0.081446,-0.003928,0.159250,7.331722
02_d_M_m_1.CEL,2.704369,0.343553,0.010664,1.352064,1.035336,1.603819,0.018263,2.193090,3.294476,3.480972,...,0.321549,0.005660,-0.332726,0.168073,0.157446,0.007049,-0.053373,-0.103942,0.150266,7.602800
03_a_M_f_1.CEL,2.292782,0.323082,0.112840,1.139095,0.894466,1.092865,-0.006393,1.665438,2.619054,2.756610,...,0.193615,-0.016850,-0.309735,0.055131,0.164192,-0.027766,-0.057326,-0.031748,0.116840,7.363773


In [91]:
michigan_diff = michigan.diff().iloc[1::2,:]
michigan_diff.index = "patient" + michigan_diff.index.str[:2]
michigan_diff.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient01,0.999779,0.394824,0.271074,0.472982,0.298698,0.477668,0.207057,1.144579,0.440795,1.369066,...,0.048478,0.033440,-0.083160,0.030243,-0.107254,0.019725,-0.007675,-0.092305,0.080254,0.378795
patient02,0.543430,0.091073,-0.094375,0.230958,0.134952,0.531185,-0.103202,0.540298,0.637883,0.911847,...,0.141391,-0.020382,-0.077449,0.063275,-0.031340,0.080924,0.028073,-0.100013,-0.008984,0.271077
patient03,0.270554,-0.066746,-0.102352,0.187250,0.042173,0.329378,0.089136,0.422810,0.640892,0.530118,...,0.027104,-0.025143,-0.098707,-0.025800,-0.121025,-0.041441,0.031566,0.001636,0.008239,0.286222
patient04,0.660731,0.221251,0.044027,0.270411,0.048237,0.140261,0.056456,1.085683,0.398471,0.866960,...,0.027601,0.006653,-0.095700,0.030953,-0.049022,-0.034425,0.037990,-0.040653,0.037088,0.681965
patient05,0.388227,0.019146,0.068072,-0.238376,-0.147348,0.233513,-0.030011,-0.194102,0.142438,-0.025070,...,-0.021588,-0.051362,0.115165,-0.084323,-0.130298,0.011086,-0.052302,0.044745,-0.003529,-0.480925


In [92]:
irvine = expression[(expression.index.str[5] == 'I') & (expression.index.str[3] != 'c') ]
irvine = irvine.sub(irvine[genes_c.index].iloc[:, -5:-1].mean(axis =1), axis = 0) 
irvine.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
01_a_I_f_2.CEL,1.774381,0.299178,0.076988,0.620824,0.456095,0.689141,-0.006664,1.423759,2.349268,0.690514,...,0.210301,-0.088819,-0.210719,-0.038234,-0.018811,-0.089101,-0.026561,0.100240,0.015422,6.337802
01_d_I_f_2.CEL,2.122828,0.381495,0.000379,0.664955,0.551410,0.879679,-0.073274,1.827232,2.202625,1.231752,...,0.230666,-0.113414,-0.194942,-0.011290,-0.003360,-0.063237,-0.044006,-0.001122,0.108365,6.223585
02_a_I_m_2.CEL,1.986773,0.235289,-0.054087,0.675169,0.554459,1.046888,-0.108865,1.694972,2.612624,1.707980,...,0.044134,-0.206595,-0.263247,-0.064663,0.016166,-0.076708,-0.129066,0.152855,0.052918,6.194819
02_d_I_m_2.CEL,2.284764,0.296745,0.018186,0.805033,0.539407,1.203011,-0.059902,1.896985,2.313536,2.268961,...,0.812011,0.509059,0.274701,0.582298,0.460916,-0.046201,-0.115016,0.063454,0.097764,5.555059
03_a_I_f_2.CEL,2.373038,0.396197,0.131910,0.936173,0.715919,1.243881,0.058697,2.102097,2.574710,2.191931,...,0.099604,-0.004565,-0.173467,0.060279,0.106888,-0.042375,-0.026153,0.034323,0.034205,6.113193


In [93]:
irvine_diff = irvine.diff().iloc[1::2,:]
irvine_diff.index = "patient" + irvine_diff.index.str[:2]
irvine_diff.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient01,0.348447,0.082317,-0.076608,0.044131,0.095315,0.190538,-0.066610,0.403473,-0.146643,0.541238,...,0.020366,-0.024594,0.015777,0.026944,0.015451,0.025864,-0.017445,-0.101362,0.092943,-0.114217
patient02,0.297991,0.061457,0.072273,0.129864,-0.015053,0.156123,0.048963,0.202013,-0.299088,0.560981,...,0.767877,0.715654,0.537948,0.646961,0.444750,0.030506,0.014050,-0.089402,0.044846,-0.639760
patient03,-0.184028,-0.055642,-0.147633,-0.264655,-0.175071,-0.103944,-0.054643,-0.533950,0.179526,-0.275680,...,-0.040773,-0.133239,0.024789,-0.049991,-0.072651,-0.032149,-0.006246,0.036402,0.001993,-0.042130
patient04,-0.581629,-0.301868,-0.193228,-0.287341,-0.219830,-0.561105,-0.150775,-0.633427,-0.209825,-0.642009,...,-0.025798,-0.101168,0.110084,-0.127972,-0.002425,0.034929,-0.006867,-0.069139,0.041077,0.227752
patient05,-0.035653,-0.003932,0.136084,0.161842,0.099975,0.269650,0.098378,-0.373250,0.365908,0.180939,...,-0.025343,0.009036,0.039556,-0.084829,-0.001845,0.019869,0.026109,-0.078807,0.032829,-0.031824


Take absolute values of each difference and compute the average over all patients for each probe sets and obtain top 20 probe sets with highest magnitude.

In [ ]:
top_probes_michigan = michigan_diff.abs().mean().sort_values(ascending = False).iloc[:100]
top_probes_michigan

38308_g_at    1.700100
33925_at      1.581787
33942_s_at    1.529290
36608_at      1.494925
35742_at      1.435630
                ...   
41498_at      1.140076
38708_at      1.137271
32315_at      1.136706
40512_at      1.134779
35174_i_at    1.133433
Length: 100, dtype: float64

Map top probe sets to top genes using `genes` dataset. 

In [ ]:
top_genes_michigan = genes.loc[top_probes_michigan.keys()].loc[:, 'sym'].values
top_genes_michigan

['NCDN', 'NRGN', 'STXBP1', 'MDH1', 'C16orf45', ..., 'HNRNPK', 'NPTN', 'FAM127A', 'OLFM1', 'VAMP2']
Length: 20
Categories (8783, object): ['AADAC', 'AAK1', 'AAMP', 'AANAT', ..., 'ZYX', 'ZZEF1', 'ZZZ3', 'psiTPTE22']

In [55]:
def get_top_genes(lab):
  if(lab == 'M' or lab == 'I'):
    df = expression[(expression.index.str[5] == lab) & (expression.index.str[3] != 'c')]
    df_diff = df.diff().iloc[1::2,:]
    control_mean = df_diff[genes_c.index].iloc[:,-5:-1].apply(np.mean, axis=1)
    control_std = df_diff[genes_c.index].iloc[:, -5:-1].apply(np.std, axis = 1)
    df_diff = df_diff.sub(control_mean, axis='rows')
    df_diff = df_diff.div(control_std, axis='rows') 
  
  else:
    df = expression[(expression.index.str[5] == lab) & (expression.index.str[3] != 'c') & ~(expression.index.str[1].isin(['1', '2', '5', '6', '7', '9'])) ]
    df_diff = df.diff().iloc[1::2,:]
  top_probes = df_diff.abs().mean().sort_values(ascending = False).iloc[:20]
  top_genes = genes.loc[top_probes.keys()].loc[:, 'sym'].values
  return np.array(top_genes)


In [56]:
top_genes_michigan = get_top_genes("M")
top_genes_irvine = get_top_genes("I")
#top_genes_davis = get_top_genes("D")
print(top_genes_irvine)
print(top_genes_michigan)
#print(top_genes_davis)

['NEFH' 'TF' 'NEFM' 'APOE' 'PLP1' 'SCN1B' 'VIM' 'C1orf61' 'CRYAB' 'SPP1'
 'MYH11' 'GFAP' 'NCDN' 'CRYAB' 'CABP1' 'HTRA1' 'ACTA2' 'NEFL' 'NTRK2'
 'GPRC5B']
['NCDN' 'NRGN' 'MDH1' 'STXBP1' 'RPL21' 'EEF1A2' 'CLASP2' 'VSNL1' 'AP2M1'
 'CBX6' 'TSPYL4' 'NPTN' 'GNAS' 'PFTK1' 'C16orf45' 'NCKAP1' 'CALM2' 'DNM1'
 'KCNJ4' 'NEFL']


## Hypothesis testing 
Wilcoxon signed-rank test for each gene(Remove controls).  
Tests whether the median of the difference is 0.  
A non-parametric alternative test to the paired t-test. 

In [ ]:
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
# Remove controls(Probesets starting with 'AFFX')
michigan_diff = michigan_diff.filter(regex='^(?!AFFX).*$', axis=1)
irvine_diff = irvine_diff.filter(regex='^(?!AFFX).*$', axis=1)


p_val_M = michigan_diff.apply(wilcoxon).iloc[1,:]
print('Michigan')
print('rejected H0(bonferroni):{}'.format(sum(multipletests(p_val_M, method='bonferroni')[0])))
print('rejected H0(BH):{} \n'.format(sum(multipletests(p_val_M, method='fdr_bh')[0])))


p_val_I = irvine_diff.apply(wilcoxon).iloc[1,:]
print('Irvine')
print('rejected H0(bonferroni):{}'.format(sum(multipletests(p_val_I, method='bonferroni')[0])))
print('rejected H0(BH):{} \n'.format(sum(multipletests(p_val_I, method='fdr_bh')[0])))

Michigan
rejected H0(bonferroni):0
rejected H0(BH):11874 

Irvine
rejected H0(bonferroni):0
rejected H0(BH):0 



In [ ]:
print(p_val_M.sort_values(ascending = True).iloc[:100])

1000_at       0.001953
35802_at      0.001953
35801_at      0.001953
39994_at      0.001953
357_at        0.001953
                ...   
40034_r_at    0.001953
35700_at      0.001953
35731_at      0.001953
35732_at      0.001953
35734_at      0.001953
Name: 1, Length: 100, dtype: float64


In [ ]:
print(p_val_I.sort_values(ascending = True).iloc[:100])

39031_at      0.013672
35457_at      0.019531
34281_at      0.037109
40202_at      0.048828
31997_at      0.048828
                ...   
39790_at      0.130859
36759_at      0.130859
2007_g_at     0.130859
34938_i_at    0.130859
1606_at       0.130859
Name: 1, Length: 100, dtype: float64
